Приведите по 2 примера, когда лучше максимизировать Precision, а когда Recall.

Почему мы используем F-меру, почему, например, нельзя просто взять среднее от Precision и Recall?

*Реализовать функции для подсчета Accuracy, Precision, Recall, F-score, которые на вход принимают y_true (истинные значения), y_pred (предсказанные значения), а на выход дается метрика.

## Задание 1

Precision: когда ложный рерузьтат недопустим

1) При отправке писем, если одно сообщение имеет цену, то важно выбрать людей которые откликнутся на предложение, и даже если не все потенциальные клиенты откликнутся, это менее страшно чем потратить деньги на клиентов которым предложение не выгодно(то, что мы предлагаем бесплатный продукт)

2) при пукупке дорогостоящего оборудования, закупить проверенное, но точно подходящее под производство

Recall: когда ложное срабатывание не критично

1) При покупке продуктов, нужно купить 200 фруктов, а  из них нужно 150 хороших, то лучше взять 200 и пулучить 30 плохих чем, если есть система отбора качества продуктов взять 150 и получить 2 плохих

2) при исследовании популяции птиц редкого вида, лучше охватить больший круг похожий и потом сделать дополнительную выборку

## Задание 2

F-score - метрика объединяющая в себе точность и полноту, выдавая как результат их гармоническое среднее, а коэффициент  𝛽 , позволяет балансировать вклад, вносимый точностью, тем самым акцентируя внимание на той метрике, максимизация которой для нас наиболее приоритетна.

Если взять простое среднее, тогда мы можем каждая метрика будет равноценной при анализе, что является некорректной предпосылкой для решения многих задач где одна метрика важнее другой. Регулировка коэффициент  𝛽 позволяет выставлять веса и регулировать важность метрик  

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (roc_auc_score, roc_curve, auc, confusion_matrix, \
                             accuracy_score, classification_report, plot_confusion_matrix, \
                             plot_precision_recall_curve, precision_recall_curve, recall_score,
                             plot_roc_curve)

# чтение данных

In [4]:
DATASET_PATH = 'training_project_data.csv'
PREP_DATASET_PATH = 'training_project_data_prep.csv'

In [5]:
df = pd.read_csv(DATASET_PATH)

In [6]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT
0,150000.0,2,2,2,24,1,2,0,0,0,...,17853.0,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,30748.0,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,6784.0,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0
3,150000.0,2,2,2,35,0,0,0,0,0,...,55182.0,48983.0,50136.0,3500.0,2000.0,20000.0,2000.0,2000.0,2000.0,0
4,70000.0,2,1,1,35,1,2,2,2,2,...,47026.0,45971.0,47573.0,0.0,6500.0,1800.0,0.0,2500.0,2000.0,0


In [7]:
for colname in ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
    df[colname] = df[colname].astype(str)

In [8]:
df.dtypes

LIMIT_BAL             float64
SEX                    object
EDUCATION              object
MARRIAGE               object
AGE                     int64
PAY_1                  object
PAY_2                  object
PAY_3                  object
PAY_4                  object
PAY_5                  object
PAY_6                  object
BILL_AMT1             float64
BILL_AMT2             float64
BILL_AMT3             float64
BILL_AMT4             float64
BILL_AMT5             float64
BILL_AMT6             float64
PAY_AMT1              float64
PAY_AMT2              float64
PAY_AMT3              float64
PAY_AMT4              float64
PAY_AMT5              float64
PAY_AMT6              float64
NEXT_MONTH_DEFAULT      int64
dtype: object

In [9]:
for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

SEX

2    6068
1    3932
Name: SEX, dtype: int64
****************************************************************************************************

EDUCATION

2    4678
1    3496
3    1674
5      90
4      42
6      18
0       2
Name: EDUCATION, dtype: int64
****************************************************************************************************

MARRIAGE

2    5332
1    4547
3     108
0      13
Name: MARRIAGE, dtype: int64
****************************************************************************************************

PAY_1

0     4895
-1    2852
1     1217
2      881
3      155
Name: PAY_1, dtype: int64
****************************************************************************************************

PAY_2

0     5178
-1    3318
2     1318
3      175
1       11
Name: PAY_2, dtype: int64
****************************************************************************************************

PAY_3

0     5200
-1    3379
2     1288
3      131
1        2
Name: PAY_3, 

# Обработка выбросов

In [10]:
df.loc[df['EDUCATION'] == '0', 'EDUCATION'] = df['EDUCATION'].mode()[0]

In [11]:
df.loc[df['MARRIAGE'] == '0', 'MARRIAGE'] = df['MARRIAGE'].mode()[0]

# построение новых признаков

In [12]:
df['IS_MALE'] = df['SEX'].map({'1':'1', '2':'0'}).astype(int)

In [13]:
for cat_colname in df.select_dtypes(include='object').columns[1:]:
    df = pd.concat([df, pd.get_dummies(df[cat_colname], prefix=cat_colname)], axis=1)

In [14]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,PAY_4_2,PAY_4_3,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3
0,150000.0,2,2,2,24,1,2,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,0,1,0,0,1,0,0,1,0,0
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,0,0,0,1,0,0,0,1,0,0
3,150000.0,2,2,2,35,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,70000.0,2,1,1,35,1,2,2,2,2,...,1,0,0,0,1,0,0,1,0,0


# Сохранение обучающего датасета

In [15]:
df.to_csv(PREP_DATASET_PATH, index=False, encoding='utf-8')

# обучение модели

In [16]:
X = df.drop(columns='NEXT_MONTH_DEFAULT')
y = df['NEXT_MONTH_DEFAULT']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=2)
X_train.shape, X_test.shape

((7000, 60), (3000, 60))

In [18]:
%%time
tree = DecisionTreeClassifier(random_state=1)

param_grid=[{
             'max_depth': range(1, 7), 
             'min_samples_leaf': range(1, 7), 
            }]

grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
tree = grid_search.best_estimator_

Wall time: 14.5 s


In [19]:
grid_search.best_params_

{'max_depth': 3, 'min_samples_leaf': 6}

In [20]:
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

pred_proba_test = tree.predict_proba(X_test)
pred_proba_test[:5]

array([[0.80078637, 0.19921363],
       [0.80078637, 0.19921363],
       [0.89638205, 0.10361795],
       [0.89638205, 0.10361795],
       [0.89638205, 0.10361795]])

# подсчет метрик

In [21]:
from sklearn.metrics import precision_score, \
    recall_score, accuracy_score, f1_score

In [38]:
def calc_score_train(y_train, pred_train):
    accuracy_train = accuracy_score(y_train, pred_train)
    
    precision_train = precision_score(y_train, pred_train)
    
    f1_score_train = f1_score(y_train, pred_train)
    
    recall_score_train = recall_score(y_train, pred_train)
    score = {
        'accuracy': accuracy_train,
        'precision': precision_train,
        'f1': f1_score_train,
        'recall': recall_score_train
    }
    for item, value in score.items():
        print(item + '= ' + str(value) + '\n\n' + '*' * 50)


In [41]:
def calc_score_test(y_test, pred_test):
    accuracy_test = accuracy_score(y_test, pred_test)
    
    precision_test = precision_score(y_test, pred_test)
    
    f1_score_test = f1_score(y_test, pred_test)
    
    recall_score_test = recall_score(y_test, pred_test)
    score = {
        'accuracy': accuracy_test,
        'precision': precision_test,
        'f1': f1_score_test,
        'recall': recall_score_test
    }
    for item, value in score.items():
        print(item + '= ' + str(value) + '\n\n' + '*' * 50)

In [39]:
calc_score_train(y_train, pred_train)

accuracy= 0.8275714285714286

**************************************************
precision= 0.701067615658363

**************************************************
f1= 0.49476768522394315

**************************************************
recall= 0.38227684346701163

**************************************************


In [42]:
calc_score_test(y_test, pred_test)

accuracy= 0.821

**************************************************
precision= 0.6546961325966851

**************************************************
f1= 0.46884272997032644

**************************************************
recall= 0.36517719568567025

**************************************************
